In [3]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset,DataLoader
import torch.optim as optim
torch.manual_seed(1015)
# define 'device' to upload tensor in gpu
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
train = pd.read_csv("./data/train.csv", encoding = 'cp949')
train['DateTime'] = pd.to_datetime(train.DateTime)
train['date'] = train.DateTime.dt.date
train  = train.groupby('date').sum().reset_index()

In [5]:
# scaling
mini = train.iloc[:,1:].min()
size = train.iloc[:,1:].max() - train.iloc[:,1:].min()
train.iloc[:,1:] = (train.iloc[:,1:] -  mini) / size

input_window = 30
output_window = 7

window_x = np.zeros((train.shape[0] - (input_window + output_window), input_window, 4))
window_y = np.zeros((train.shape[0] - (input_window + output_window), output_window, 4))

for start in range(train.shape[0] - (input_window + output_window)):
    end = start + input_window    
    window_x[start,:, :] = train.iloc[start : end                , 1: ].values
    window_y[start,:, :] = train.iloc[end   : end + output_window, 1: ].values

In [6]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size = input_size,
                            hidden_size = hidden_size,
                            batch_first=True)
        self.hidden_lstm = nn.LSTM(input_size = hidden_size,
                                   hidden_size = hidden_size,
                                   batch_first=True)
        
        self.time_fc = nn.Linear(hidden_size, 4)
    
    def forward(self, x_time):
    
        out_time, _ = self.lstm(x_time)
        out_time, _ = self.hidden_lstm(out_time)
        out_time, _ = self.hidden_lstm(out_time)
        out_time, _ = self.hidden_lstm(out_time)
        
        out_time = self.time_fc(out_time[:,-7:, :])
        
        return out_time.view(-1,7,4)
    
model = LSTM(input_size = 4, hidden_size = 30).to(device)

In [7]:
window_x = torch.tensor(window_x).float().to(device)
window_y = torch.tensor(window_y).float().to(device)

# Train model
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-2)
criterion = nn.MSELoss(size_average = True)
num_epochs  = 500
train_error = []
for t in range(num_epochs):
    train_pred = model(window_x)
    loss = criterion(train_pred, window_y) ### trend
    train_error.append(loss)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if t % 10 == 0 and t !=0:
        print(f"{t} Epochs train MSE: {loss.item():1.5f}")

10 Epochs train MSE: 0.02835
20 Epochs train MSE: 0.01247
30 Epochs train MSE: 0.01320
40 Epochs train MSE: 0.01062
50 Epochs train MSE: 0.00831
60 Epochs train MSE: 0.00823
70 Epochs train MSE: 0.00801
80 Epochs train MSE: 0.00792
90 Epochs train MSE: 0.00782
100 Epochs train MSE: 0.00774
110 Epochs train MSE: 0.00765
120 Epochs train MSE: 0.00756
130 Epochs train MSE: 0.00740
140 Epochs train MSE: 0.00702
150 Epochs train MSE: 0.00638
160 Epochs train MSE: 0.00639
170 Epochs train MSE: 0.00631
180 Epochs train MSE: 0.00607
190 Epochs train MSE: 0.00592
200 Epochs train MSE: 0.00566
210 Epochs train MSE: 0.00553
220 Epochs train MSE: 0.00542
230 Epochs train MSE: 0.00529
240 Epochs train MSE: 0.00526
250 Epochs train MSE: 0.00533
260 Epochs train MSE: 0.00498
270 Epochs train MSE: 0.00529
280 Epochs train MSE: 0.00484
290 Epochs train MSE: 0.00449
300 Epochs train MSE: 0.00422
310 Epochs train MSE: 0.00468
320 Epochs train MSE: 0.00422
330 Epochs train MSE: 0.00485
340 Epochs train MS

In [8]:
submission = pd.read_csv("./data/submission.csv", encoding = 'cp949')

In [10]:
submission = pd.read_csv("./data/submission.csv", encoding = 'cp949')

#last_month = torch.tensor(window_x[-1,:,:][np.newaxis,...]).float().to(device) <- 수정 전
last_month = train.iloc[-30:,1:].values[np.newaxis,...] # <- 수정 후
last_month = torch.tensor(last_month).float().to(device) # <- 수정 후

for start in range((len(submission) - output_window)//7 + 2):
    start = start * 7
    next_week = model(last_month)
    #last_month = torch.cat([last_month[-7:], next_week], axis = 1) <- 수정 전
    last_month = torch.cat([last_month[:,7:,:], next_week], axis = 1)# <- 수정 후

    pred_week = next_week.cpu().detach().numpy().reshape(output_window,4)
    pred_week = pred_week * size.values + mini.values
    pred_week = pred_week.astype(int)
    
    if start/7 == (len(submission) - output_window)//7 + 1:
        submission.iloc[start :, 1:] = pred_week[-submission.iloc[start :, 1:].shape[0]:,:]
    else:
        submission.iloc[start : start + output_window, 1:] = pred_week
submission

,DateTime,사용자,세션,신규방문자,페이지뷰
0,2020-11-09,3642,3693,908,95140
1,2020-11-10,3044,3019,724,77261
2,2020-11-11,2773,2742,653,66082
3,2020-11-12,2729,2694,649,66466
4,2020-11-13,2484,2453,567,62954
...,...,...,...,...,...
56,2021-01-04,3139,3067,710,75017
57,2021-01-05,3691,3608,925,89935
58,2021-01-06,2332,2318,554,57258
59,2021-01-07,1819,1794,384,41891


In [ ]:
submission.to_csv('submission_pre.csv', index = False, encoding = 'cp949')